# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
os.chdir('..')
sys.path.append('src')

In [ ]:
from PIL import Image
from pathlib import Path
from functools import partial, reduce
from collections import defaultdict
import multiprocessing as mp
from contextlib import contextmanager

import cv2
import numpy as np
from tqdm.auto import tqdm


import utils
import data
from sampler import GdalSampler

# Code

## Narezator

In [ ]:
@contextmanager
def poolcontext(*args, **kwargs):
    pool = mp.Pool(*args, **kwargs)
    yield pool
    pool.terminate()
    
def mp_func(foo, args, n):
    args_chunks = [args[i:i + n] for i in range(0, len(args), n)]
    with poolcontext(processes=n) as pool:
        pool.map(foo, args_chunks)
    
def mp_foo(foo, args): return foo(*args)

In [ ]:
def mp_sampler(dst, i_fn, m_fn, a_fn, wh, wh_mask, idxs):
    s = GdalSampler(i_fn, m_fn, a_fn, wh, wh_mask)
    for idx in idxs:
        i,m = s[idx]  
        m*=255
        
        img_dir = dst / 'imgs' / i_fn.with_suffix('').name
        os.makedirs(str(img_dir), exist_ok=True)
        
        mask_dir = dst / 'masks' / i_fn.with_suffix('').name
        os.makedirs(str(mask_dir), exist_ok=True)
        
        # TODO ZFILL NAME
        orig_name = (str(idx).zfill(6) + '.png')
        img_name = img_dir / orig_name 
        mask_name = mask_dir /orig_name
        
        #print(mask_name, m.shape, m.dtype)
        
        cv2.imwrite(str(img_name), i.transpose(1,2,0))
        cv2.imwrite(str(mask_name), np.expand_dims(m,-1).repeat(3,-1))

In [ ]:
p = Path('input/hm/train')
#p = Path('/home/sokolov/work/webinf/data/kidney/train/')
dst_path = Path('input/train')
NUM_PROC = 4
wh = (512,512)

In [ ]:
filt = partial(utils.filter_ban_str_in_name, bans=['-', '_ell'])
ann_fns = sorted(utils.get_filenames(p, '*.json', filt))
masks_fns = sorted(utils.get_filenames(p, '*_mask.tiff', filt))
img_fns = sorted([a.with_suffix('.tiff') for a in ann_fns])
img_fns, ann_fns, masks_fns

In [ ]:
#assert  False , 'DO ONCE'
# for i_fn, m_fn, a_fn in tqdm(zip(img_fns, masks_fns, ann_fns)):
#     const_args = i_fn, m_fn, a_fn, wh, wh
#     _s = GdalSampler(*const_args)
#     part_samp = partial(mp_sampler, *(dst_path, *const_args))
#     mp_func(part_samp, range(len(_s)), NUM_PROC)
#     #break

## Datasets

In [ ]:
root = Path('input/cuts512')
sd = data.SegmentDataset(root / 'imgs', root / 'masks', mode_train=False)
len(sd)

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact(idx=(0, len(sd)),continuous_update=False)
def view(idx): return sd._view(idx)

## Dataloaders

In [ ]:
from config import cfg, cfg_init
from pprint import pprint

In [ ]:
pprint(cfg)

In [ ]:
cfg_init('src/configs/unet.yaml')
cfg['TRANSFORMERS']['TRAIN']['AUG'] = 'light'

In [ ]:
datasets = data.build_datasets(cfg)

In [ ]:
tds = datasets['TRAIN']

In [ ]:
i,m = tds[0]

In [ ]:
i.shape, i.dtype, i.max(), i.mean(), i.std()

In [ ]:
m.shape, m.dtype, m.max()#, m.mean(), m.std()

In [ ]:

dls = data.build_dataloaders(cfg, datasets, pin=True, drop_last=False)

In [ ]:
tdl = dls['TRAIN']
for xb, yb in tdl:
    break

In [ ]:
xb.shape, xb.dtype, xb.mean(), xb.std()

In [ ]:
yb.shape, yb.dtype, yb.max()